In [13]:
using GamsStructure

include("verify_data.jl")

sum_wrong (generic function with 1 method)

In [4]:
GU = load_universe("../data/nationaldata_ls_julia");
GU[:i][:use].active = false
GU[:i][:oth].active = false

GU[:j][:use].active = false
GU[:j][:oth].active = false

false

In [5]:
gams_GU = GamsUniverse()
set_dir = "../data/nationaldata_ls_gams/set"
@GamsSets(gams_GU,set_dir,begin
    :i, "lkajdf"
    :fd, ""
    :m, ""
    :r, ""
    :ts, ""
    :va, ""
    :yr, ""
end);
alias(gams_GU,:i,:j);

parm_dir = "../data/nationaldata_ls_gams/parameter"
@GamsParameters(gams_GU,parm_dir,begin
    :y_0, (:yr,:i), ""
    :ys_0, (:yr,:j,:i), ""
    :ty_0, (:yr,:j), ""
    :id_0, (:yr,:i,:j), ""
    :fd_0, (:yr,:i,:fd), ""
    :va_0, (:yr,:va,:j), ""
    :m_0, (:yr,:i), ""
    :x_0, (:yr,:i), ""
    :ms_0, (:yr,:i,:m), ""
    :md_0, (:yr,:m,:i), ""
    :s_0, (:yr,:i), ""
    :a_0, (:yr,:i), ""
    :bopdef_0, (:yr,), ""
    :ta_0, (:yr,:i), ""
    :tm_0, (:yr,:i), ""
end);

In [6]:
wrong = wrong_entries(GU,gams_GU)

Dict{Any, Any}()

In [14]:
verify_key(:y0,wrong,GU,gams_GU)

In [12]:
sum_wrong(GU,gams_GU)

a_0 -> 0.00044234955753191807
id_0 -> 0.0015479466060224492
ys_0 -> 0.000687914148466751
ms_0 -> 7.973968729384131e-5
x_0 -> 0.0
s_0 -> 0.0005105541485086462
tm_0 -> 0.0
va_0 -> 0.0003371160986776478
fd_0 -> 0.0006189685090511636
m_0 -> 2.0539125955565396e-15
md_0 -> 6.131010887542725e-5
ty_0 -> 0.0
bopdef_0 -> 0.0
ta_0 -> 0.0
y_0 -> 0.0004063173317255253


In [ ]:
wrong = Dict()

key = :y0

for key in keys(gams_GU.parameters)

    old_parm = gams_GU[key]
    parm = GU[key]
    sets = [[e for e in gams_GU[s]] for s in old_parm.sets]
    for idx in Iterators.product(sets...)
        ind = [[e] for e in idx]
        if !(isapprox(old_parm[ind...],parm[ind...],atol = 1e-5))
            if key ∈ keys(wrong)
                push!(wrong[key],ind)
            else
                wrong[key] = [ind]
            end
        end
    end
end

for (key,p) in wrong
    println("$key -> $(length(p))")
end

In [ ]:
key = :y_0
out = ""
for (key,p) in wrong
    out*="$key\n\n"
    for idx in p
        out*="$idx -> $(GU[key][idx...]) | $(gams_GU[key][idx...])\n"
    end
    out*="\n-----------\n"
end
print(out)